In [ ]:
from notebook.services.config import ConfigManager
from traitlets.config.manager import BaseJSONConfigManager
# set jupyter configurations
path = "/home/stefan/.jupyter/nbconfig" # update with your path
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
              'theme': 'white',
              'transition': 'fade',
              'start_slideshow_at': 'selected',
                'center' : False,
                'width' : 1024,
                'height': 768,   
                'scroll': True,
                'slideNumber': True,
})
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulating Waves

- Let's say, we are investigating sine waves
- Interested in finding amplitude $\theta$ given an observed wave $x$
    - Posterior $p(\theta | x)$
- We can implicitly define likelihood $p(x | \theta)$ by building simulator
- Let's have a look at this simulator

In [ ]:
def wave_simulator(theta, plot=False):
    """ f(x | theta) = theta * sin(x) """
    xs = np.linspace(0,10,100) # x values
    observation = theta * torch.sin(torch.Tensor(xs))  + torch.randn(len(xs)) * 0.1 # y values
    if plot: # make it usable in interactive plot
        plt.plot(observation,label=f'$\\theta={theta}$',color='#FF1053')
        plt.ylim(-5, 5)
        plt.legend()
        plt.show()

    return observation

You can play around with the simulator and produce different observations by adjusting the $\theta$ parameter.

In [ ]:
simulation = interactive(wave_simulator, theta=(-2.0, 2.0), plot=True)
simulation

- Use simulator to produce many observations $x$ and form synthetic dataset
- Choose sample size $n$ 
- Define uniform distribution to represent prior belive about what $\theta$ could be
- Let `sbi` do the work

In [ ]:
simulation_hyperparameters = interactive(lambda sample_size, prior_lower, prior_upper: (sample_size, prior_lower, prior_upper),
                          sample_size=(0,1000),prior_lower=(-2,-0.1),prior_upper=(0.1,2))

simulation_hyperparameters

In [ ]:
# initiate samples 
num_samples, prior_lower, prior_upper = simulation_hyperparameters.result

samples = np.zeros((num_samples, len(np.linspace(0,10,100))))

# create prior
prior = utils.BoxUniform(low=prior_lower*torch.ones(1), high=prior_upper*torch.ones(1))

# run simulator and infer 
posterior = infer(wave_simulator, prior, method='SNPE', num_simulations=num_samples)

- Great! We now have a posterior over $\theta$
- Let's have a look whether it is good
- Use simulator to create another example observation
- See whether probability mass accumulates at chosen $\theta$

In [ ]:
 simulation

In [ ]:
sample = posterior.sample((10000,), x=simulation.result)

_ = analysis.pairplot(sample, limits=[[-2,2],[-2,2],[-2,2]], figsize=(6,6))

Are you satisfied with the conditional posterior? If not, maybe, try out different hyperparameters.